In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_synergistic_nudge_evolutionary as synergistic_nudge_ev

In [3]:
#distribution parameters
input_variables = 4
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


start_mutation_size = nudge_size/4,
change_mutation_size = nudge_size/(4*10)
mutations_per_update_step = 10
synergetic_nudge = synergistic_nudge_ev.SynergisticNudge(input_dist, np.copy(input_dist), cond_output,
                 nudge_size, mutations_per_update_step, start_mutation_size,
                 change_mutation_size, timestamp=0)

In [5]:
synergistic_nudge_ev.TEST = False
input_distribution = np.copy(input_dist)
conditional_output_distribution = np.copy(cond_output)
nudge_size = 0.01
evolutionary_parameters = {
    "number_of_generations": 100,
    "population_size": 10,
    "number_of_children": 20, 
    "generational": True,
    "mutation_size": nudge_size/4,
    "parent_selection_mode": "rank_exponential",
    "start_mutation_size": nudge_size/25,
    "change_mutation_size": nudge_size/(25*10),
    "mutations_per_update_step": 10
}

max_synergistic_nudge = synergistic_nudge_ev.find_synergistic_nudge_with_max_impact(
    input_distribution, conditional_output_distribution, nudge_size, 
    evolutionary_parameters
)


In [ ]:
NUDGE_SIZE = 0.01
new_distribution = max_synergistic_nudge.new_distribution
l1_norm_to_old_distribution = np.sum(np.absolute(input_dist-max_synergistic_nudge.new_distribution))
print(l1_norm_to_old_distribution)
actual_new_distribution = input_dist + (new_distribution-input_dist)*(NUDGE_SIZE/l1_norm_to_old_distribution)

In [ ]:

for i in range(len(new_distribution.shape)):
    new_marginal = probability_distributions.ProbabilityArray(actual_new_distribution).marginalize(set([i]))
    old_marginal = probability_distributions.ProbabilityArray(input_dist).marginalize(set([i]))
    print(np.sum(np.absolute(new_marginal-old_marginal)))

In [ ]:
import nudge_non_causal
print("according to the score {}".format(max_synergistic_nudge.score))
nudge_non_causal.find_nudge_impact(input_dist, actual_new_distribution, cond_output, measure="absolute")